In [35]:
# Bin Packing Problem
# Involves packing items into bins with a different objective
# Find the fewest bins that will hold all the items

# the differences between the two problems:
# Multiple knapsack problem: Pack a subset of the items into a fixed number of bins, with varying capacities, 
#so that the total value of the packed items is a maximum.
# Bin packing problem: Given as many bins with a common capacity as necessary, find the fewest that will hold all the items. 
#In this problem, the items aren't assigned values, because the objective doesn't involve value.

#In this example, items of various weights need to be packed into a set of bins with a common capacity. 
#Assuming that there are enough bins to hold all the items, 
#the problem is to find the fewest that will suffice.

# Import the libraries
import docplex.mp

# first import the Model class from docplex.mp
from docplex.mp.model import Model
from docplex.mp.linear import LinearExpr

In [36]:
#Create the data
#The data includes the following:
# weights: A vector containing the weights of the items.
# bin_capacity: A single number giving the capacity of the bins.
# There are no values assigned to the items because the goal of minimizing the number of bins doesn't involve value.
# num_bins is set to the number of items. 
# This is because if the problem has a solution, then the weight of every item must be less than or equal to the bin capacity. 
# In that case, the maximum number of bins you could need is the number of items, because you could always put each item in a separate bin.
def create_data_model(num_weights, bin_capacity):
    import random
    data = {}
    weights = [random.randint(19,48) for i in range(num_weights)]
    #weights = [48, 30, 19, 36, 36, 27, 42, 42, 36, 24, 30]
    data['weights'] = weights
    data['items'] = list(range(len(weights)))
    #num_bins = 5
    data['bins'] = data['items']
    data['bin_capacity'] = bin_capacity
    return data

data = create_data_model(4, 60)

In [37]:
# create one model instance, with a name
m = Model(name='AMC Bin Packing Problem')

# Variables
# x[i, j] = 1 if item i is packed in bin j.
x = {}
for i in data['items']:
    for j in data['bins']:
        x[i, j] = m.binary_var(name='x_%i_%i'%(i,j))

# y[j] = 1 if bin j is used, i.e., if any items are packed in it — and 0 otherwise. The sum of the y[j] will be the number of bins used.
y = {}
for j in data['bins']:
    y[j] = m.binary_var(name='y_%i'%j)

        
# Constraints
# Each item must be in exactly one bin.
#This constraint is set by requiring the sum of x[i][j] over all bins j to be equal to 1.
for i in data['items']:
    m.add_constraint(m.sum([x[i, j] for j in data['bins']]) == 1)

# The total weight packed in each bin cannot exceed its capacity.
# This constraint is set by requiring the sum of the weights of items placed in bin j to be less than or equal to 
# the product of the bin variable and the capacity of the bin.
# we multiply the bin capacity on the right side of the inequalities by y[j] - 
# because it forces y[j] to equal 1 if any item is packed in bin j. 
#This is so because if y[j] were 0, the right side of the inequality would be 0, while the bin weight on the left side would be greater than 0, 
#violating the constraint. 
#This connects the variables y[j] to the objective of the problem, for now the solver will try to minimize the number of bins for which y[j] is 1.
for j in data['bins']:
    m.add_constraint(m.sum([x[i, j] * data['weights'][i] for i in data['items']]) <= y[j] * data['bin_capacity'])

# Objective: minimize the number of bins used.
m.minimize(m.sum([y[j] for j in data['bins']]))

print(m.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Bin Packing Problem

Minimize
 obj: y_0 + y_1 + y_2 + y_3
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 + x_0_3 = 1
 c2: x_1_0 + x_1_1 + x_1_2 + x_1_3 = 1
 c3: x_2_0 + x_2_1 + x_2_2 + x_2_3 = 1
 c4: x_3_0 + x_3_1 + x_3_2 + x_3_3 = 1
 c5: 30 x_0_0 + 26 x_1_0 + 36 x_2_0 + 21 x_3_0 - 60 y_0 <= 0
 c6: 30 x_0_1 + 26 x_1_1 + 36 x_2_1 + 21 x_3_1 - 60 y_1 <= 0
 c7: 30 x_0_2 + 26 x_1_2 + 36 x_2_2 + 21 x_3_2 - 60 y_2 <= 0
 c8: 30 x_0_3 + 26 x_1_3 + 36 x_2_3 + 21 x_3_3 - 60 y_3 <= 0

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_0_3 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_1_3 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1
 0 <= x_2_3 <= 1
 0 <= x_3_0 <= 1
 0 <= x_3_1 <= 1
 0 <= x_3_2 <= 1
 0 <= x_3_3 <= 1
 0 <= y_0 <= 1
 0 <= y_1 <= 1
 0 <= y_2 <= 1
 0 <= y_3 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_0_3 x_1_0 x_1_1 x_1_2 x_1_3 x_2_0 x_2_1 x_2_2 x_2_3 x_3_0
 x_3_1 x_3_2 x_3_3 

In [38]:
result = m.solve()

In [39]:
print(result)

solution for: AMC Bin Packing Problem
objective: 2
x_0_1=1
x_1_1=1
x_2_0=1
x_3_0=1
y_0=1
y_1=1



In [40]:
print('No of bins used:', result.get_objective_value())
num_bins = 0.
for j in data['bins']:
    if result.get_value('y_%i'%j) == 1:
        bin_items = []
        bin_weight = 0
        for i in data['items']:
            if result.get_value('x_%i_%i'%(i,j)) > 0 :
                bin_items.append(i)
                bin_weight += data['weights'][i]
        if bin_weight > 0:
            num_bins += 1
            print('Bin number', j)
            print('  Items packed:', bin_items)
            print('  Total weight:', bin_weight)
            print()
print()
print('Number of bins used:', num_bins)
print(result.solve_details)


No of bins used: 2.0
Bin number 0
  Items packed: [2, 3]
  Total weight: 57

Bin number 1
  Items packed: [0, 1]
  Total weight: 56


Number of bins used: 2.0
status  = integer optimal solution
time    = 0.125 s.
problem = MILP
gap     = 0%



In [41]:
m.export_as_lp(path='503-Bin Packing Problem.lp')

'503-Bin Packing Problem.lp'

In [42]:
m.print_information()

Model: AMC Bin Packing Problem
 - number of variables: 20
   - binary=20, integer=0, continuous=0
 - number of constraints: 8
   - linear=8
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
